In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

In [2]:
df = pd.read_csv('../data/raw/all_crashes.csv')
df.head()

,geom,Num_Occupants,Num_MotorVehicles,Fatalities_calc,SuspectedSeriousInjury_calc,NonMotorist_calc,Aggregated_Non_Motorist_txt,CityTownship,CountyNameTxt,Region,...,LocalCodeTxt,doi_date,doi_time,doi_dow_txtCARTO,HitAndRunInd_txtCARTO,VehicleMakeModelYearTxt_list,VehicleTypeCde_txtCARTO_list,doi_hour_txtCARTO,CrashType,CrashDetail
0,POINT(-93.38856788 45.28870554),7,2,1,1,0,Motorist Only,ANDOVER,ANOKA,Metro,...,25217514,"September 30, 2025",06:59:00 AM,Tuesday,"No, Did not Leave Scene","2022 GMC SIERRA, 2015 UNKNOWN 3000","Pickup, School Bus",6:00 AM - 6:59 AM,Collision w/ Non-Fixed Object,Motor Vehicle in Transport
1,POINT(-93.37732941 45.25945286),2,2,0,0,0,Motorist Only,ANDOVER,ANOKA,Metro,...,25217042,"September 30, 2025",02:55:00 PM,Tuesday,"No, Did not Leave Scene","2015 CHEVROLET MALIBU, 2004 CHEVROLET SILVERADO","Passenger Car, Pickup",2:00 PM - 2:59 PM,Collision w/ Non-Fixed Object,Motor Vehicle in Transport
2,POINT(-94.14949353 45.12032389),1,1,0,0,0,Motorist Only,COKATO,WRIGHT,East Central,...,25027310,"September 30, 2025",06:58:00 PM,Tuesday,"No, Did not Leave Scene",2017 CHEVROLET EQUINOX,Sport Utility Vehicle,6:00 PM - 6:59 PM,Non-Collision,Overturn/Rollover
3,POINT(-94.14932826 45.1203201),1,1,0,0,0,Motorist Only,COKATO,WRIGHT,East Central,...,WP25027349,"September 30, 2025",07:20:00 AM,Tuesday,"No, Did not Leave Scene",2017 CHEVROLET BOLT EV,Sport Utility Vehicle,7:00 AM - 7:59 AM,Collision w/ Fixed Object,Standing Tree/Shrubbery
4,POINT(-93.31679869 45.18055865),2,2,0,0,0,Motorist Only,COON RAPIDS,ANOKA,Metro,...,25217663,"September 30, 2025",10:52:00 AM,Tuesday,"No, Did not Leave Scene","2025 TESLA MODEL 3, 2014 BMW 535","Passenger Car, Passenger Car",10:00 AM - 10:59 AM,Collision w/ Non-Fixed Object,Motor Vehicle in Transport


In [3]:
from shapely import wkt
from src.preprocessing import generate_grid

df['geometry'] = df['geom'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

grid = generate_grid(gdf)
grid.head()

,cell_id,geometry
0,0_0,"POLYGON ((-97.19939 43.85696, -97.13696 43.857..."
1,0_1,"POLYGON ((-97.20018 43.90178, -97.13771 43.902..."
2,0_2,"POLYGON ((-97.20097 43.9466, -97.13845 43.9471..."
3,0_3,"POLYGON ((-97.20176 43.99143, -97.1392 43.9919..."
4,0_4,"POLYGON ((-97.20255 44.03627, -97.13995 44.036..."


In [4]:
from src.preprocessing import create_cell_day_df

cell_day_df = create_cell_day_df(gdf, grid, '2025-09-01', '2025-09-30')
cell_day_df.head()

,cell_id,date,num_crashes,crash_tomorrow
0,0_0,2025-09-01,0,0
1,0_0,2025-09-02,0,0
2,0_0,2025-09-03,0,0
3,0_0,2025-09-04,0,0
4,0_0,2025-09-05,0,0


In [6]:
cell_day_df.info()
cell_day_df.describe()
print(cell_day_df['num_crashes'].value_counts())
print(cell_day_df['crash_tomorrow'].mean())

<class 'pandas.core.frame.DataFrame'>
Index: 393530 entries, 0 to 34048
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   cell_id         393530 non-null  object        
 1   date            393530 non-null  datetime64[ns]
 2   num_crashes     393530 non-null  int64         
 3   crash_tomorrow  393530 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 15.0+ MB
num_crashes
0     390195
1       2466
2        524
3        196
4         67
5         45
6         13
7         10
8          5
9          3
10         3
11         2
12         1
Name: count, dtype: int64
0.00853556272711102


In [7]:
pd.crosstab(cell_day_df['cell_id'], cell_day_df['crash_tomorrow'])



crash_tomorrow,0,1
cell_id,,
0_0,29,0
0_1,29,0
0_10,29,0
0_100,29,0
0_101,29,0
...,...,...
9_95,29,0
9_96,29,0
9_97,29,0
